# View this notebook here:
https://nbviewer.jupyter.org/github/zagoodman/microeconomics_videos/blob/master/jupyter/assemble_data.ipynb

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prep" data-toc-modified-id="Prep-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prep</a></span></li><li><span><a href="#Merge-anonymized-data" data-toc-modified-id="Merge-anonymized-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Merge anonymized data</a></span><ul class="toc-item"><li><span><a href="#Demographic-data" data-toc-modified-id="Demographic-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Demographic data</a></span><ul class="toc-item"><li><span><a href="#Preceding-term-vars" data-toc-modified-id="Preceding-term-vars-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Preceding term vars</a></span></li><li><span><a href="#Student-level" data-toc-modified-id="Student-level-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Student-level</a></span></li><li><span><a href="#GPA-and-course-units" data-toc-modified-id="GPA-and-course-units-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>GPA and course units</a></span><ul class="toc-item"><li><span><a href="#Concurrent-term" data-toc-modified-id="Concurrent-term-2.1.3.1"><span class="toc-item-num">2.1.3.1&nbsp;&nbsp;</span>Concurrent term</a></span></li><li><span><a href="#Following-term" data-toc-modified-id="Following-term-2.1.3.2"><span class="toc-item-num">2.1.3.2&nbsp;&nbsp;</span>Following term</a></span></li></ul></li></ul></li><li><span><a href="#Treatment-and-outcome-data" data-toc-modified-id="Treatment-and-outcome-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Treatment and outcome data</a></span><ul class="toc-item"><li><span><a href="#Student-level" data-toc-modified-id="Student-level-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Student level</a></span></li></ul></li></ul></li><li><span><a href="#Videos" data-toc-modified-id="Videos-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Videos</a></span><ul class="toc-item"><li><span><a href="#100A" data-toc-modified-id="100A-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>100A</a></span></li><li><span><a href="#Export" data-toc-modified-id="Export-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Export</a></span></li></ul></li></ul></div>

This file takes anonymized data from the T+LC, cleans them, and returns analysis-ready data frames:
1. `id-year_level_data.csv`: treatment and exam score data
2. `dem_concurrent.csv`: demographic data and GPA/courseload during the quarter of the experiment
3. `dem_nextquarter.csv`: demographic data and GPA/courseload during the quarter following treatment

All code in this file is in Python 3.

## Prep

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
from matplotlib import pyplot as plt

In [ ]:
import pandas_flavor as pf

@pf.register_dataframe_accessor('z')
class MyFlavor(object):
    
    def __init__(self, df):
        self._df = df

    def get_unique_students(self, var1 = "id", var2 = "year"):
        df = self._df
        x = len(df[[var1, var2]].drop_duplicates())
        print("N unique students: {}".format(x))
        return x
    
    def get_vars(self):
        df = self._df
        cols = [x for x in df if x in tlcvarnames]
        return df[cols].rename(columns = tlcvarnames)
    
tlcvarnames = {'id': 'id',
               'year': 'year',
               'de id': 'id', 
               'deid': 'id',
               'previous_cum_gpa': 'prev_cumgpa',
               'term_code_econ': 'term',
               'term-econ100a': 'term',
               'term code courses': 'term_courses',
               'term_precoursegpa': 'term_pregpa',
               'term code econ 100a': 'termecon',
               'apct_type_desc': 'apptype',
               'ethnicity_grouped': 'ethnicity',
               'gender': 'gender',
               'term_code_econ': 'term',
               'year_econ': 'year',
               'year - econ 100a': 'year',
               'year - econ100a': 'year',
               'year-econ100a': 'year',
               'year-zack': 'year',
               'measure names': 'measure',
               'measure values': 'values',
               'Class Units - Letter Grade': 'units_letter',
               'Class Units - P/NP': 'units_pnp',
               'Class Units - Withdrawn': 'units_w',
               'GPA - Classes Letter Grade': 'gpa_letter',
               'GPA - Classes Letter Grade - No Econ': 'gpa_letter_sansecon',
               'GPA - Classes Letter Grade - No Econ 100A': 'gpa_letter_sans100a',
               'GPA - Classes Letter Grade - Only Econ - No Econ 100A': 'gpa_econ_sans100a',
               'N Classes - Letter Grade': 'nclass_letter',
               'N Classes - Not Passed': 'nclass_np',
               'N Classes - P/NP': 'nclass_pnp',
               'N Classes - Passed': 'nclass_p',
               'N Classes - Withdrawn': 'nclass_w'
              }

## Merge anonymized data

### Demographic data

#### Preceding term vars

In [ ]:
# pre 100A (term before) cumulative gpa. Missing for all entering freshmen or first time UCSD students

dfd = pd.read_excel("../data/raw/Econ-Goodman-Su20-Pre Course GPA-FA18.xlsx")
dfd = pd.concat([dfd, pd.read_excel("../data/raw/Econ-Goodman-Su20-Pre Course GPA-FA19.xlsx")], 0)
dfd.columns = [x.lower() for x in dfd.columns]
dfd = dfd.z.get_vars()
dfd.loc[dfd.term == 'FA18', 'year'] = 2018
dfd.loc[dfd.term == 'FA19', 'year'] = 2019
# reorder and drop term
dfd = dfd[['id', 'year', 'term_pregpa', 'prev_cumgpa']]

print(len(dfd))
dfd.z.get_unique_students(var2='id')
display(dfd.isnull().sum())
#dfd.head()

#### Student-level

In [ ]:
# ethnicity, gender, transfer status

df = pd.read_excel("../data/raw/Econ-Goodman-Su20-Student-level.xlsx")
df.rename(str.lower, axis=1, inplace=True)
df = df.z.get_vars()
df['transfer'] = np.array(df.apptype == 'Transfer Student', dtype='int')
df.loc[df.gender == 'Men (Cis & Trans)', 'gender'] = 'm'
df.loc[df.gender == 'Women (Cis & Trans)', 'gender'] = 'f'
df.loc[df.gender == 'Data Unavailable', 'gender'] = 'u'
# drop term and reorder
df = df[['id', 'year', 'ethnicity', 'gender', 'transfer', 'apptype']]

# merge with dfd
dfd = df.merge(dfd, how='outer', on=['id', 'year'])

print(len(dfd))
dfd.z.get_unique_students()
print(dfd.isnull().sum())
#dfd.head()

#### GPA and course units

In [ ]:
df = pd.read_excel("../data/raw/Econ-Goodman-Su20-Student-quarter-level_FA18-WI19.xlsx")
df.columns = [x.lower() for x in df.columns]
df = df.z.get_vars()
df2 = pd.read_excel("../data/raw/Econ-Goodman-Su20-Student-quarter-level_FA19-WI20.xlsx")
df2.columns = [x.lower() for x in df2.columns]
df2 = df2.z.get_vars()

df = pd.concat([df, df2], 0)
print(len(df))
display(pd.crosstab(df.year, df.term))

##### Concurrent term

In [ ]:
## concurrent data

dfc = df.loc[df.term_courses.isin(["FA18", "FA19"])]
dfc = dfc[['id', 'year', 'measure', 'values']]

# reshape wide
dfc = dfc.pivot_table(index=['id', 'year'], columns='measure', values='values').reset_index()
dfc = dfc.z.get_vars()

# merge with rest of demographic data
dfd = dfd.merge(dfc, how='outer', on=['id', 'year'])

# add zeros where needed
for x in ['units_pnp', 'units_w']:
    dfd.loc[dfd[x].isnull(), x] = 0

# check and explore data
print(len(dfd))
dfd.z.get_unique_students()
print(dfd.dtypes)
print(dfd.isnull().sum())

# export
dfd.to_csv("../data/generated/dem_concurrent.csv", index=False)

##### Following term

In [ ]:
## following quarter's data

dff = df.loc[df.term_courses.isin(["WI19", "WI20"])]
dff = dff[['id', 'year', 'measure', 'values']]

# reshape wide
dff = dff.pivot_table(index=['id', 'year'], columns='measure', values='values').reset_index()
dff = dff.z.get_vars()

# merge with rest of demographic data
dfd = dfd.iloc[:, 0:8]
dfd = dfd.merge(dff, how='outer', on=['id', 'year'])

# add zeros where needed
for x in ['units_letter', 'units_pnp', 'units_w', \
          'nclass_letter', 'nclass_np', 'nclass_pnp', \
          'nclass_p', 'nclass_w']:
    dfd.loc[dfd[x].isnull(), x] = 0

# check and explore data
print(len(dfd))
dfd.z.get_unique_students()
print(dfd.dtypes)
print(dfd.isnull().sum())

# export
dfd.to_csv("../data/generated/dem_nextquarter.csv", index=False)

### Treatment and outcome data

#### Student level

In [ ]:
df = pd.read_excel("../data/raw/DeID_all_pid-year-level_data with description.xlsx")
df.rename(columns={'DeID': 'id'}, inplace=True)
print(len(df))
df.z.get_unique_students()
df = df[['id'] + [x for x in df.columns if x != 'id']]
#df.head()

In [ ]:
# I think that's all I need for now...may add more later

# drop the two duplicate entries
print(len(df))
df = df.drop_duplicates()
print(len(df))
df.z.get_unique_students()

# export
df.to_csv("../data/generated/id-year_level_data.csv", index=False)

## Videos

### 100A

In [ ]:
dfv = pd.read_csv('../data/raw/DeID_video-level_data.csv')
dfv = dfv.loc[dfv.videoid.notnull()]
dfv.loc[dfv.syllabus_week.isnull(), 'syllabus_week'] = -1

# int columns
for v in dfv.columns[7:-1]:
    dfv[v] = dfv[v].apply(lambda x: int(x))
    
# date columns
dfv['lastview'] = pd.to_datetime(dfv['last.view'])
dfv['firstview'] = pd.to_datetime(dfv['first.view'])

# unnecessary cols
dfv.drop(['first.view', 'last.view', 'videocode', 'length_mins'], 1, inplace=True)

print(dfv.isnull().sum())
dfv.head(2)

In [ ]:
# Explore

dftmp = dfv[['syllabus_week', 'incentivized', 'videoid']].drop_duplicates()
print('This many relevant videos: {}'.format(sum(dftmp.syllabus_week.notnull())))
print('This many incentivized videos: {}'.format(sum(dftmp.incentivized > 0)))
print('\nThis many relevant videos by syllabus week:')
print(dftmp.syllabus_week.value_counts().sort_index())
print('\nThis many students per year:')
print(dfv[['DeID', 'year']].drop_duplicates().year.value_counts())

In [ ]:
# duration per video (seconds)

# Steps:
# 1. Get duration from lastview and firstview
# 2. Topcode duration by the length of the video
# 3. Impute duration when = 0 by multiplying max duration by within-student average % watched

# Step 1
dfv['duration'] = dfv.apply(lambda x: (x['lastview'] - x['firstview']).seconds, 1)
dfv['duration_max'] = dfv.minutes * 60 + dfv.seconds

# Step 2
dfv['duration'] = dfv.apply(lambda x: min(x.duration, x.duration_max), 1)
plt.hist(dfv.duration, 50)
plt.title('Distribution of video durations (seconds)')
plt.show()
plt.hist(dfv.duration / dfv.duration_max, 50)
plt.title('Distribution of video durations (percent)')
plt.show()

# Step 3
dfv['duration_impute'] = np.array(dfv.duration == 0, dtype='int')
dfv['percent'] = dfv.duration / dfv.duration_max
dfv['percent'] = dfv.groupby(['DeID', 'duration_impute'])['percent'].transform('mean')

dfsub = dfv.loc[dfv.duration_impute == 0, ['DeID', 'percent']].drop_duplicates()
dfsub.columns = ['DeID', 'p_duration']
plt.hist(dfsub.p_duration, 50)
plt.title('Dist. of mean % watched, 0\'s excluded')
plt.show()

# merge dfsub on dfv and impute
dfv = dfv.merge(dfsub, on=['DeID'], how='left')
dfv.loc[dfv.p_duration.isnull(), 'p_duration'] = 0.521 # see next cell
dfv.loc[dfv.duration_impute == 1, 'duration'] = dfv.loc[dfv.duration_impute == 1].\
    apply(lambda x: x.p_duration * x.duration_max, 1)
plt.hist(dfv.duration, 50)
plt.title('Distribution of imputed and observed vid duration (seconds)')
plt.show()
plt.hist(dfv.duration / dfv.duration_max, 50)
plt.title('Distribution of imputed and observed duration (%)')
plt.show()

In [ ]:
# Aside - look at relationship between having lots of videos at 0s and % needing imputation, N videos

# def plot function
from statsmodels.nonparametric.smoothers_lowess import lowess

def plot_lowess(x, y, title=''):
    """Plots scatter of y vs x with lowess overlaid. Give y and x pre sorted.
    """
    lowess_y = lowess(y, x, frac=0.25, is_sorted=True, return_sorted=False, missing='raise')
    plt.figure(figsize=(8,4))
    plt.scatter(x, y)
    plt.plot(x, lowess_y, 'red')
    plt.title(title)
    plt.show()

    
# prep data
dftmp = dfv.groupby('DeID')['duration_impute'].agg(['mean']).reset_index()
dftmp = dftmp.merge(dfv.loc[dfv.duration_impute == 0, ['DeID', 'p_duration']].drop_duplicates(), 
                   on='DeID', how='left')
dftmp.columns = ['DeID', 'p_impute', 'p_duration']

# is there a relationship between % watched and % needing imputation?
dfsub = dftmp.loc[dftmp.p_duration.notnull()].copy().sort_values('p_impute')
plot_lowess(dfsub.p_impute, dfsub.p_duration, '% needing imputation vs mean % duration')
                  
# Do those with high imputation watch different number of videos?
dftmp2 = dfv.groupby('DeID').videoid.agg(['count']).reset_index()
dftmp2.columns = ['DeID', 'vcount']
dftmp = dftmp.merge(dftmp2, on='DeID', how='left')
dfsub = dftmp.loc[dftmp.vcount.notnull() & (dftmp.vcount < 300)].copy().sort_values('p_impute')
plot_lowess(dfsub.p_impute, dfsub.vcount, '% needing imputation vs N videos watched')

# mean duration vs N videos watched
dfsub = dftmp.loc[dftmp.p_duration.notnull() & (dftmp.vcount < 300)].copy().sort_values('vcount')
plot_lowess(dfsub.vcount, dfsub.p_duration,  'N videos watched vs mean % duration')


# Look at those with no imputation info
dfsub = dftmp.loc[dftmp.p_impute == 1]
print('This many have no duration info: {}'.format(len(dfsub.DeID.unique())))
print('Watched this many vids on average: {}'.format(dfsub.vcount.mean()))
scalar = dftmp.loc[(dftmp.vcount <= 3) | (dftmp.p_impute > 0.9)].p_duration.mean()
print('Avg. duration for those who watched <3 vids or need >90% imputed: {:.3f}%'.format(scalar))

In [ ]:
# gen helpful flags
from datetime import datetime as dt

# first flag
dfv['firstflag'] = dfv.groupby(['DeID', 'year', 'videoid']).firstview.transform('min') == dfv.firstview

# before midterm 1
dfv['b4_mid1'] = ((dfv.year == 2018) & (dfv.firstview <= dt(2018, 10, 19, 19, 20))) | \
                 ((dfv.year == 2019) & (dfv.firstview <= dt(2019, 10, 23, 21, 20))) 

# before midterm 2
dfv['b4_mid2'] = ((dfv.year == 2018) & (dfv.firstview <= dt(2018, 11, 19, 19, 20))) | \
                 ((dfv.year == 2019) & (dfv.firstview <= dt(2019, 11, 13, 21, 20))) 

# before final
dfv['b4_final'] = ((dfv.year == 2018) & (dfv.firstview <= dt(2018, 12, 8, 18, 0))) | \
                 ((dfv.year == 2019) & (dfv.firstview <= dt(2019, 12, 7, 14, 30))) 

for v in ['b4_mid1', 'b4_mid2', 'b4_final']:
    print(dfv[v].value_counts())

In [ ]:
# sum duration for each student: total duration, and total unique duration. 
# only keep course relevant videos.
# do this for: pre mid1, pre mid2, pre final

# init df at DeID-level
mergevars = ['DeID', 'year']
dfd = dfv.loc[:, mergevars].drop_duplicates()

# relevant only
print('This many videos were irrelevant: {}'.format(sum(dfv.relevant_100a == 0)))
print('This percent of videos were irrelevant: {}'.format(sum(dfv.relevant_100a == 0) / len(dfv)))
dfr = dfv.loc[dfv.relevant_100a > 0]

# Avg unique videos watched - imputed vs not
u_imputed = dfr.groupby('DeID').videoid.agg('nunique').mean()
u_n_imputed = dfr.loc[dfr.duration_impute == 0].groupby('DeID').videoid.agg('nunique').mean()
print('This percent of unique videos imputed: {}'.format((u_imputed - u_n_imputed) / u_imputed))
print('This percent of video counts imputed: {}'.format(sum(dfr.duration_impute)/len(dfr)))

# relevant duration, all (not unique) - before mid1, mid2, final
dfagg = dfr.copy()
dfagg['duration_mid1'] = dfagg.duration * dfagg.b4_mid1
dfagg['duration_mid2'] = dfagg.duration * dfagg.b4_mid2
dfagg['duration_final'] = dfagg.duration * dfagg.b4_final
dfaggd = dfagg.groupby(mergevars)[['duration_mid1', 'duration_mid2', 'duration_final']].agg('sum').reset_index()
dfd = dfd.merge(dfaggd, on=mergevars, how='left')

# relevant duration, unique - before mid1, mid2, final
# keep only max duration per videoid
dfaggd = dfagg.groupby(['DeID', 'year', 'videoid'])[['duration_mid1', 'duration_mid2', 'duration_final']].agg('max').reset_index()
dfaggd = dfaggd.groupby(mergevars)[['duration_mid1', 'duration_mid2', 'duration_final']].agg('sum').reset_index()
dfaggd.columns = ['DeID', 'year', 'duration_mid1_u', 'duration_mid2_u', 'duration_final_u']
dfd = dfd.merge(dfaggd, on=mergevars, how='left')

# units of hours
for v in dfd.columns[2:]:
    dfd[v] = dfd[v] / 3600

# sanity check
print('Sanity checks: should be 0 first.')
print(sum(dfd.duration_final < dfd.duration_final_u - .01))
print(sum(dfd.duration_final <= dfd.duration_final_u + .01))

# do the same thing for counts

# count all
dfaggd = dfr.groupby(mergevars)[['b4_mid1', 'b4_mid2', 'b4_final']].agg('sum').reset_index()
dfaggd.columns = ['DeID', 'year', 'videos_mid1', 'videos_mid2', 'videos_final']
dfd = dfd.merge(dfaggd, on=mergevars, how='left')

# count unique
dfaggd = dfr.copy()
for v in ['mid1', 'mid2', 'final']:
    dfaggd['videos_'+v+'_u'] = dfaggd.loc[dfaggd['b4_'+v] == 1, 'videoid']
dfaggd = dfaggd.groupby(mergevars)[['videos_mid1_u', 'videos_mid2_u', 'videos_final_u']].agg('nunique').reset_index()
dfd = dfd.merge(dfaggd, on=mergevars, how='left')

# sanity check
print('Sanity checks: should be 0 first.')
print(sum(dfd.videos_final < dfd.videos_final_u))
print(sum(dfd.videos_final <= dfd.videos_final_u))

# rename deid to id
dfd.rename(columns={'DeID': 'id'}, inplace=True)

display(dfd.head(3))

### Export

In [ ]:
# export video data by updating the pid-year-level data

# TODO: add 100B stuff

df = pd.read_csv("../data/generated/id-year_level_data.csv")
dropvars = ['videos', 'relevant', 'videos_u', 'relevant_u', \
            'duration_all', 'duration_rel', 'duration_u', \
            'videos_b4_mid1_rel', 'videos_b4_mid1_relu', \
            'videos_b4_mid2_rel', 'videos_b4_mid2_relu']

df = df.loc[:, [c for c in df.columns if c not in dropvars]]
df.head()

# merge dfd on df and add zeros when no videos observed
df = df.merge(dfd, on=['id', 'year'], how='outer')
for c in df.columns[-12:]:
    df.loc[df[c].isnull(), c] = 0
for c in df.columns[-6:]:
    df[c] = df[c].apply(lambda x: int(x))

# print(df.isnull().sum())
# display(df.head())

# save csv
df.to_csv("../data/generated/id-year_level_data.csv", index=False)

In [ ]:
# When do students watch videos?
plt.hist(dfv.firstview.apply(lambda x: x.hour), 24)